# 출력층 설계 (Output layer)

In [1]:
!pip3 install torch torchvision torchaudio

### 소프트맥스 오버플로우 방지

In [2]:
import numpy as np

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

def stable_softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

x = np.array([1000, 1001, 1002])

softmax(x), stable_softmax(x)

/var/folders/qv/kf5vm6ln5k113k6v46_6ynn00000gn/T/ipykernel_16460/1534837267.py:4: RuntimeWarning: overflow encountered in exp
  exp_x = np.exp(x)
/var/folders/qv/kf5vm6ln5k113k6v46_6ynn00000gn/T/ipykernel_16460/1534837267.py:5: RuntimeWarning: invalid value encountered in divide
  return exp_x / np.sum(exp_x)


(array([nan, nan, nan]), array([0.09003057, 0.24472847, 0.66524096]))

In [3]:
x = np.array([2.0, 1.0, 0.1])
stable_softmax(x)

array([0.65900114, 0.24243297, 0.09856589])

- pytorch 라이브러리 함수 사용

In [4]:
import torch
import torch.nn.functional as F

# x = torch.tensor([2.0, 1.0, 0.1], dtype=torch.float32)
x = torch.tensor([1000, 1001, 1002], dtype=torch.float32)

softmax_output = F.softmax(x, dim=0) # 내부적으로 stable softmax를 사용
softmax_output   


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/envs/pystudy_env/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/pystudy_env/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/envs/pystudy_env/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.

tensor([0.0900, 0.2447, 0.6652])

In [5]:
sigmoid_output = torch.sigmoid(x)
sigmoid_output

tensor([1., 1., 1.])

## 손실 함수와 연계

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

# 간단한 다중 클래스 분류 모델 정의 (입력 값: 5개, 출력: 3 클래스)
class SimpleMultiClassModel(nn.Module):
    def __init__(self):
        super(SimpleMultiClassModel, self).__init__()
        self.fc = nn.Linear(5, 3)
        
    def forward(self, x):
        return self.fc(x)
    
model = SimpleMultiClassModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 데이터 생성
inputs = torch.randn(4, 5)
labels = torch.tensor([0, 2, 1, 0])

for _ in range(10):
    outputs = model(inputs) # 순전파
    loss = criterion(outputs, labels) # 손실 계산
    print(loss.item())
    
    optimizer.zero_grad() # 이전 단계에서 계산된 기울기를 0으로 초기화
    loss.backward() # 손실에 대한 역전파 수행
    optimizer.step() # 계산된 가중치를 사용하여 옵티마이저가 모델의 파라메터를 업데이트

1.5865001678466797
1.547620177268982
1.5093942880630493
1.4718420505523682
1.434981346130371
1.3988285064697266
1.3633983135223389
1.3287028074264526
1.2947523593902588
1.2615551948547363
